In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
from simple_cnn import SimpleCNN

In [2]:
# Transformations: Resize, Normalize
transform_main_model = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to fixed size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [ ]:
# --- Dataset base (senza transform) ---
dataset = ImageFolder(root='data')  # nessuna transform qui

# --- Carica lo split ---
split = torch.load('splits/dataset_split.pth')
train_indices = split['train_indices']
val_indices = split['val_indices']

# --- Applica lo split ---
train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)

# --- Wrapper per applicare transform dinamicamente ---
class TransformedSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        return self.transform(img), label

    def __len__(self):
        return len(self.subset)

# --- Applica le trasformazioni specifiche ---
train_data = TransformedSubset(train_subset, transform_main_model)

# --- Calcola il numero di classi a partire dal training set ---
all_labels = [label for _, label in train_subset]
num_classes = len(set(all_labels))

print(f"Numero di classi (utenti autorizzati): {num_classes}")

val_data = TransformedSubset(val_subset, transform_main_model)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [27]:
# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN(num_classes=num_classes).to(device)

In [28]:
# Step 3: Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
# Step 4: Train the Model
train_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    # Validation Accuracy
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)
    val_accuracies.append(100 * correct_val / total_val)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracies[-1]:.2f}%, Validation Accuracy: {val_accuracies[-1]:.2f}%")

Epoch [1/25], Loss: 6.5989, Train Accuracy: 4.71%, Validation Accuracy: 2.60%
Epoch [2/25], Loss: 5.3329, Train Accuracy: 10.56%, Validation Accuracy: 5.52%
Epoch [3/25], Loss: 4.3105, Train Accuracy: 18.68%, Validation Accuracy: 14.29%
Epoch [4/25], Loss: 3.3496, Train Accuracy: 31.52%, Validation Accuracy: 20.78%
Epoch [5/25], Loss: 2.4802, Train Accuracy: 44.92%, Validation Accuracy: 30.52%
Epoch [6/25], Loss: 1.7467, Train Accuracy: 57.84%, Validation Accuracy: 39.29%
Epoch [7/25], Loss: 1.2008, Train Accuracy: 71.08%, Validation Accuracy: 23.38%
Epoch [8/25], Loss: 0.7626, Train Accuracy: 83.35%, Validation Accuracy: 47.08%
Epoch [9/25], Loss: 0.5044, Train Accuracy: 89.68%, Validation Accuracy: 40.91%
Epoch [10/25], Loss: 0.3462, Train Accuracy: 93.26%, Validation Accuracy: 28.57%
Epoch [11/25], Loss: 0.2725, Train Accuracy: 94.72%, Validation Accuracy: 27.27%
Epoch [12/25], Loss: 0.2063, Train Accuracy: 96.67%, Validation Accuracy: 45.45%
Epoch [13/25], Loss: 0.1465, Train Accur

In [12]:
torch.save(model.state_dict(), 'handwriting_identification_main_model.pth')

In [ ]:
# Step 5: Plot Training and Validation Metrics
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

# --- Inferenza con softmax per test e valutazione confidenza ---

import torch.nn.functional as F

model.eval()
all_preds = []
all_targets = []
all_confidences = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)
        confs, predicted = torch.max(probs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())
        all_confidences.extend(confs.cpu().numpy())

# Esempio: stampa prime 10 predizioni e confidenze
for i in range(10):
    print(f"Target: {all_targets[i]}, Predicted: {all_preds[i]}, Confidenza: {all_confidences[i]:.4f}")
